In [ ]:
import pandas as pd

def convert_tcga_format_fixed(column_name):
    """
    Convert TCGA format from 'tcga3za93z01' to 'TCGA.3Z.A93Z.01'
    Correct pattern analysis:
    tcga3za93z01 (12 chars) -> TCGA.3Z.A93Z.01 (15 chars)
    
    Breaking down the 12-character input:
    - tcga (4 chars) -> TCGA.
    - 3z (2 chars) -> 3Z.
    - a (1 char) -> A
    - 93z (3 chars) -> 93Z.
    - 01 (2 chars) -> 01
    """
    if column_name.startswith('tcga') and len(column_name) == 12:
        # Convert to uppercase first
        upper_col = column_name.upper()
        
        # Correct segmentation based on actual pattern
        tcga_part = upper_col[0:4]      # 'TCGA'
        project = upper_col[4:6]        # '3Z'
        tss = upper_col[6:7]            # 'A'
        participant = upper_col[7:10]   # '93Z' (3 chars, not 4!)
        sample = upper_col[10:12]       # '01'
        
        # Reconstruct with proper dots
        return f"{tcga_part}.{project}.{tss}{participant}.{sample}"
    
    return column_name

# Test with known examples
test_cases = [
    ('tcga3za93z01', 'TCGA.3Z.A93Z.01'),
    ('tcga6daa2e01', 'TCGA.6D.AA2E.01'),
    ('tcgaa3335701', 'TCGA.A3.3357.01')
]

print("Testing FIXED conversion function:")
for input_col, expected in test_cases:
    result = convert_tcga_format_fixed(input_col)
    match = "✓" if result == expected else "✗"
    print(f"{match} {input_col} -> {result} (expected: {expected})")

# If all tests pass, re-clean the tables
all_passed = all(convert_tcga_format_fixed(input_col) == expected for input_col, expected in test_cases)

if all_passed:
    print("\n✓ All tests pass! Re-cleaning tables with correct function...")
    
    # List of tables to clean
    tables_to_clean = [
        ('KIRC_CNV_top', '/workdir/KIRC_CNV_top.csv'),
        ('KIRC_Methy_top', '/workdir/KIRC_Methy_top.csv'),
        ('KIRC_mRNA_top', '/workdir/KIRC_mRNA_top.csv')
    ]
    
    for table_name, file_path in tables_to_clean:
        print(f"\nRe-processing {table_name}...")
        
        # Load the original table
        df = pd.read_csv(file_path)
        print(f"  Shape: {df.shape}")
        
        # Get current column names
        original_columns = df.columns.tolist()
        
        # Convert column names with fixed function
        new_columns = [convert_tcga_format_fixed(col) for col in original_columns]
        
        # Count changes
        changes = sum(1 for orig, new in zip(original_columns, new_columns) if orig != new)
        print(f"  Converted {changes} column names")
        
        # Apply new column names
        df.columns = new_columns
        
        # Save cleaned table (overwrite the previous versions)
        output_path = f"/workdir/execution_outputs/{table_name}_column_cleaned.csv"
        df.to_csv(output_path, index=False)
        print(f"  Updated cleaned table: {output_path}")
    
    print("\nFinal verification:")
    # Load survival data for verification
    survival_df = pd.read_csv("/workdir/survival_KIRC.csv")
    survival_samples = set(survival_df['sample_name'].tolist())
    
    # Check one cleaned table
    cnv_cleaned = pd.read_csv("/workdir/execution_outputs/KIRC_CNV_top_column_cleaned.csv")
    cnv_patient_cols = [col for col in cnv_cleaned.columns if col.startswith('TCGA')][:10]
    
    matches = sum(1 for col in cnv_patient_cols if col in survival_samples)
    print(f"Verification: {matches}/{len(cnv_patient_cols)} column names match survival sample names")
    print("Sample cleaned column names:", cnv_patient_cols[:5])
    
else:
    print("\n✗ Tests failed. Need to debug further.")
